# Call of Duty League Prediction Model

In [ ]:
#Load all the imports and file 
!pip install scikit-learn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 2.4 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.1 MB 5.6 MB/s eta 0:00:02
   ---------------------------------------- 11.1/11.1 MB 21.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   ------------- -------------------------- 15.2/44.8 MB 86.9 MB/s eta 0:00:01
   ------------------ --------------------- 20.2/44.8 MB 49.0 MB/s eta 0:00:01
   --------------------------------- ------ 38.0/44.8 MB 61.9 MB/s eta 0:00:01
   ---------------------------------------- 44.8/44.8 MB 56.9 MB/s eta 0:00:00
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [24]:
# Import necessary libraries
import pandas as pd

# Clean the data again (if necessary)
cleaned_dataframes = {name: clean_data(df) for name, df in dataframes.items()}

# Display the cleaned dataframes for review
def display_dataframe_to_user(title, dataframe):
    """Displays a dataframe with a title for context."""
    print(f"\n{title}:\n{'=' * len(title)}")
    print(dataframe.head())  # Display the first 5 rows for review

# Display the dataframes
display_dataframe_to_user("Call of Duty League Stats (Cleaned)", cleaned_dataframes['league_stats'])
display_dataframe_to_user("Player Stats (Cleaned)", cleaned_dataframes['player_stats'])
display_dataframe_to_user("Roster Data (Cleaned)", cleaned_dataframes['roster'])
display_dataframe_to_user("Team Stats (Cleaned)", cleaned_dataframes['team_stats'])

NameError: name 'cleaned_dataframes' is not defined

In [21]:
Ace_tools Error# Re-import the necessary tools library

import ace_tools as tools

# Clean the data again (if necessary) and display it
cleaned_dataframes = {name: clean_data(df) for name, df in dataframes.items()}

# Display the cleaned dataframes for review
tools.display_dataframe_to_user("Call of Duty League Stats (Cleaned)", cleaned_dataframes['league_stats'])
tools.display_dataframe_to_user("Player Stats (Cleaned)", cleaned_dataframes['player_stats'])
tools.display_dataframe_to_user("Roster Data (Cleaned)", cleaned_dataframes['roster'])
tools.display_dataframe_to_user("Team Stats (Cleaned)", cleaned_dataframes['team_stats'])


ModuleNotFoundError: No module named 'ace_cleaning'

In [26]:
# Rename the "ID" column in the roster dataframe to "Player" to match the player stats and team stats datasets
roster_df = roster_df.rename(columns={"ID": "Player"})

# Strip whitespace and clean the "Player" column in all datasets
roster_df["Player"] = roster_df["Player"].str.strip()
cleaned_dataframes['player_stats']["Player"] = cleaned_dataframes['player_stats']["Player"].str.strip()

# Merge the roster and player stats dataframes using the "Player" column
merged_roster_stats = pd.merge(
    roster_df,
    cleaned_dataframes['player_stats'],
    on="Player",
    how="inner"
)

# Display the merged dataframe for user review
tools.display_dataframe_to_user("Merged Roster and Player Stats (Using Player ID)", merged_roster_stats)


NameError: name 'roster_df' is not defined

# Initial Cleaning and Preparation:
Removed "?" and Unnecessary Symbols: Cleaned the raw data by replacing placeholders like ? with meaningful values or NaNs and removing irrelevant symbols.
Combined Rows for Headers: Merged the first two rows in the dataset to create a proper header row.
Aligned Columns and Data: Adjusted data under the headers to ensure alignment with the correct columns.
Column Adjustments:
Renamed Columns:
Changed "Overall Player" to "ID" for consistency between datasets.
Removed "Unnamed" Columns: Eliminated columns with ambiguous names like "Unnamed" that didn't contribute meaningful data.
Removed Extra Columns: Deleted irrelevant columns such as "Team" and "NaN" after merging to streamline the dataset.
Data Sorting and Merging:
Sorted Alphabetically:
Sorted the "ID" column in both the roster and league stats datasets alphabetically to ensure proper alignment before merging.
Merged Datasets: Combined the sorted roster and league stats datasets on the "ID" column to create a unified table.
Final Adjustments:
Reformatted Data: Reset indices and ensured all data rows aligned under the correct headers after merging.
Reviewed and Verified Data: Checked for inconsistencies, ensuring the final dataset was clean, sorted, and free of redundant columns.
The result is a well-organized table that aligns player-specific stats and details accurately from both datasets.

## Now we have the player stats and the Teams they are on. To predict a match between teams take the player stats of each team excluding Hardpoint D, Seach Destroy D, and Overall D and Control D and add them. The higher the total in each category the more likely they are to win

In [25]:
# Group by "Current Team" and sum the relevant player stats for each team
team_stats = merged_data.groupby('Current Team')[relevant_columns].sum()

# Display the team-wise aggregated stats for review
tools.display_dataframe_to_user("Team Aggregated Stats (Without Excluded Columns)", team_stats)

NameError: name 'merged_data' is not defined

In [30]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Load the dataset with team averages
all_teams_combined = pd.read_csv("Merged_Roster_and_League_Stats_Data__Team_and_NaN_Columns_Removed_.csv")

# Filter numeric columns for modeling
numeric_columns = all_teams_combined.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Generate synthetic matchups for training
matchup_data = []
teams = all_teams_combined['Current Team'].unique()
for i in range(len(teams)):
    for j in range(i + 1, len(teams)):
        # Extract the averaged stats for both teams
        team_a = all_teams_combined[all_teams_combined['Current Team'] == teams[i]].iloc[-1][numeric_columns].values
        team_b = all_teams_combined[all_teams_combined['Current Team'] == teams[j]].iloc[-1][numeric_columns].values
        # Create matchups with the difference in stats
        matchup_data.append((team_a - team_b, 1))  # Team A wins
        matchup_data.append((team_b - team_a, 0))  # Team B wins

# Prepare the feature set and labels
matchup_features = [x[0] for x in matchup_data]
matchup_labels = [x[1] for x in matchup_data]

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(matchup_features)
y = matchup_labels

# Train a logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X, y)

# Function to calculate win probability for a specific matchup
def calculate_win_probability(team1, team2):
    # Retrieve the stats for both teams
    team1_stats = all_teams_combined[all_teams_combined['Current Team'] == team1].iloc[-1][numeric_columns].values
    team2_stats = all_teams_combined[all_teams_combined['Current Team'] == team2].iloc[-1][numeric_columns].values
    # Compute the difference in stats and standardize
    matchup_diff = scaler.transform([team1_stats - team2_stats])
    # Predict the win probability
    probability = model.predict_proba(matchup_diff)[0][1]  # Probability of Team 1 winning
    return probability

# Example Usage
team1 = "ATL FaZe"
team2 = "VAN Surge"
win_probability = calculate_win_probability(team1, team2)
print(f"Win Probability for {team1} against {team2}: {win_probability:.2%}")

c:\Users\camer\OneDrive\Documents\GitHub\Project1_Group3\Project1_Group3\.conda\Lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\camer\OneDrive\Documents\GitHub\Project1_Group3\Project1_Group3\.conda\Lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\camer\OneDrive\Documents\GitHub\Project1_Group3\Project1_Group3\.conda\Lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [28]:
# Graphically Show the Win Probability 
import matplotlib.pyplot as plt
import numpy as np

# Generate win probabilities for all pairwise matchups
win_probabilities = []
matchup_labels = []

for i in range(len(teams)):
    for j in range(i + 1, len(teams)):
        team1 = teams[i]
        team2 = teams[j]
        probability = calculate_win_probability(team1, team2)
        win_probabilities.append(probability)
        matchup_labels.append(f"{team1} vs {team2}")

# Sort matchups by probability for visualization
sorted_indices = np.argsort(win_probabilities)[::-1]
sorted_probabilities = [win_probabilities[i] for i in sorted_indices]
sorted_labels = [matchup_labels[i] for i in sorted_indices]

# Create a horizontal bar chart
plt.figure(figsize=(10, 8))
plt.barh(sorted_labels, sorted_probabilities, color='skyblue')
plt.xlabel("Win Probability")
plt.title("Win Probabilities for Team Matchups")
plt.gca().invert_yaxis()  # Invert y-axis to have the highest probability on top
plt.tight_layout()
plt.show()

NameError: name 'calculate_win_probability' is not defined